In [5]:
import pandas as pd
from scipy.io import arff
import numpy as np

In [39]:
data = arff.loadarff('./CM1.arff')
# data = arff.loadarff("./test.arff")
df = pd.DataFrame(data[0])
head_list = df.columns.values.tolist()

In [40]:
#标准化
data_without_YN = df.drop("Defective",axis = 1)
data_normalize = (data_without_YN-data_without_YN.mean())/(data_without_YN.std())

data_normalize['Defective'] = df.Defective


In [41]:
row_yes_data = df[df.Defective == b'Y']
row_yes_data = row_yes_data.drop("Defective",axis = 1).values
row_no_data =  df[df.Defective == b'N']
row_no_data = row_no_data.drop("Defective",axis = 1).values
row_yes_data

array([[ 15.  ,   7.  ,   3.  , ...,  67.  ,  39.22,  32.  ],
       [ 27.  ,   9.  ,   1.  , ...,  83.  ,  47.27,  33.  ],
       [155.  ,  71.  ,   5.  , ..., 536.  ,  48.42, 210.  ],
       ...,
       [ 35.  ,  21.  ,   9.  , ..., 186.  ,  45.33,  91.  ],
       [  4.  ,   3.  ,   2.  , ...,  28.  ,  56.52,  11.  ],
       [  0.  ,  11.  ,   3.  , ...,  38.  ,  24.32,  30.  ]])

In [32]:
#得到缺陷样本
yes_samples = data_normalize[data_normalize.Defective == b"Y"]
yes_samples = yes_samples.drop("Defective",axis = 1)
#得到无缺陷样本
no_samples = data_normalize[data_normalize.Defective == b"N"]
no_samples = no_samples.drop("Defective",axis = 1)
#近邻数k
k = len(no_samples)//len(yes_samples)
k

7

In [33]:
#yes样本是一个42X37的数组 存放的都是有缺陷样本
yes_samples_array = yes_samples.values
#no样本是一个302X37的数组 存放的都是没有缺陷样本
no_samples_array = no_samples.values
# 每一个yes样本和每一个no样本计算欧氏距离
# 得到的是一个二维数组 行数是yes样本的行数 列数是no样本的列数
array = [[np.sqrt(np.sum(np.square(x-y))) for y in no_samples_array]for x in yes_samples_array]
#argsort 以index的形式 从小到大排序 取7个
array = np.array(array).argsort()[:,:k]
# array = np.array(array)
#第一个yes样本和index=67的no样本,index=137个no样本....欧氏距离最短
#即找出每个yes样本的 近邻样本
array
# print(no_samples_array)

array([[ 67, 137,  89,  25, 246, 138,  10],
       [246,  67,  11, 221, 213, 150, 239],
       [214, 253, 160, 252,  73, 101, 168],
       [254,  18, 237,  31,  29, 267, 197],
       [ 97, 128, 216,  34, 243, 187, 136],
       [ 37, 255,   2, 268,  88, 269,  86],
       [246, 221,  25,  30,  70, 135,  28],
       [ 37, 298,  36, 171, 141, 159, 255],
       [ 36, 161,  28, 132, 299, 246, 229],
       [ 88,  86, 168, 268, 255,  18,   2],
       [166,  82, 219, 269, 101,  78, 268],
       [277,  93,   8,   9, 176, 297, 112],
       [118, 213,  67,  23,  97, 128, 243],
       [114,  99,  37,  88,   2, 166, 268],
       [ 17, 249, 253, 160,  73, 214, 101],
       [  2,  37, 114,  39, 237,  88, 268],
       [220, 152, 223, 155, 112,  31, 215],
       [ 20,  29,  24, 289, 218,  66, 282],
       [ 82, 101, 166, 244, 268,  86, 269],
       [301, 229, 106, 112, 167,  91, 298],
       [106,  60, 163, 215, 229, 135, 167],
       [ 23, 256,  28, 221, 145, 187,  40],
       [ 60, 223, 106, 298, 135,

In [34]:
#举例说明 不用管这里的代码
# 有缺陷样本 和 index=? 的no样本 计算特征差值
yes0 = row_yes_data[0,:]
no67 = row_no_data[0,:]
w = {i:0 for i in range(len(df))}
b = np.abs(yes0 - no67)
print(b)
#做了差以后得到要个数组 最小的值 赋的权重最低为1
#index = 2的地方的值最小  即第三个特征最小
c = pd.Series(b).rank(method='min')
# print(c)
for index in range(len(c)):
    w[index] += c[index]
print(w)
# print(b)


[9.000000e+00 2.000000e+00 1.000000e+00 0.000000e+00 1.900000e+01
 4.000000e+00 1.000000e+00 7.000000e-02 2.000000e+00 0.000000e+00
 1.000000e+00 1.000000e-01 0.000000e+00 0.000000e+00 0.000000e+00
 7.000000e+00 2.000000e+00 6.010000e+00 1.202000e+01 1.490942e+04
 1.800000e-01 7.800000e+01 6.000000e-02 8.283100e+02 5.203200e+02
 5.000000e-02 1.000000e+00 2.000000e+00 1.000000e+00 1.000000e-01
 3.200000e+01 4.600000e+01 1.700000e+01 1.200000e+01 3.500000e+01
 3.522000e+01 7.000000e+00]
{0: 25.0, 1: 17.0, 2: 12.0, 3: 1.0, 4: 29.0, 5: 21.0, 6: 12.0, 7: 8.0, 8: 17.0, 9: 1.0, 10: 12.0, 11: 9.0, 12: 1.0, 13: 1.0, 14: 1.0, 15: 23.0, 16: 17.0, 17: 22.0, 18: 27.0, 19: 37.0, 20: 11.0, 21: 34.0, 22: 7.0, 23: 36.0, 24: 35.0, 25: 6.0, 26: 12.0, 27: 17.0, 28: 12.0, 29: 10.0, 30: 30.0, 31: 33.0, 32: 28.0, 33: 26.0, 34: 31.0, 35: 32.0, 36: 23.0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 

In [36]:
#开一个字典 记录f1 f2 f3 f4...的权重
w = {i:0 for i in range(yes_samples.shape[1])}
#42个yes样本
#这里的array就是之前算的  每个yes样本 的邻近样本

for i in range(array.shape[0]):
    for j in array[i]:
        #每一行yes样本数据 - 每一行array中的no样本数据 的绝对值
        ds = np.abs(row_yes_data[i,:] - row_no_data[j,:])
        ds = pd.Series(ds).rank(method='min')
        for index in range(len(ds)):
            w[index] += ds[index]
w

{0: 7005.0,
 1: 4173.0,
 2: 3803.0,
 3: 4990.0,
 4: 7340.0,
 5: 5389.0,
 6: 3192.0,
 7: 2126.0,
 8: 3863.0,
 9: 1097.0,
 10: 3163.0,
 11: 1813.0,
 12: 6310.0,
 13: 1763.0,
 14: 1203.0,
 15: 7197.0,
 16: 2918.0,
 17: 8149.0,
 18: 6111.0,
 19: 10867.0,
 20: 2825.0,
 21: 9343.0,
 22: 1649.0,
 23: 10413.0,
 24: 10326.0,
 25: 1910.0,
 26: 2994.0,
 27: 4100.0,
 28: 5989.0,
 29: 1886.0,
 30: 8180.0,
 31: 8779.0,
 32: 7035.0,
 33: 5362.0,
 34: 8341.0,
 35: 6915.0,
 36: 7441.0}

In [46]:
#rank的举例 第0个元素排第五 第1个元素排第三
#使用min 和 不使用min的效果可以自己试一试
a = [51,22,23,14,5,5]
a = pd.Series(a)
a.rank(method='min')

0    6.0
1    4.0
2    5.0
3    3.0
4    1.0
5    1.0
dtype: float64

In [37]:

a = sorted(w.items(),key=lambda x:x[1],reverse=True)
b = [i[0] for i in a ]
c = np.array(head_list)
c = list(c[b])
print(a)
print(c)

[(19, 10867.0), (23, 10413.0), (24, 10326.0), (21, 9343.0), (31, 8779.0), (34, 8341.0), (30, 8180.0), (17, 8149.0), (36, 7441.0), (4, 7340.0), (15, 7197.0), (32, 7035.0), (0, 7005.0), (35, 6915.0), (12, 6310.0), (18, 6111.0), (28, 5989.0), (5, 5389.0), (33, 5362.0), (3, 4990.0), (1, 4173.0), (27, 4100.0), (8, 3863.0), (2, 3803.0), (6, 3192.0), (10, 3163.0), (26, 2994.0), (16, 2918.0), (20, 2825.0), (7, 2126.0), (25, 1910.0), (29, 1886.0), (11, 1813.0), (13, 1763.0), (22, 1649.0), (14, 1203.0), (9, 1097.0)]
['HALSTEAD_EFFORT', 'HALSTEAD_PROG_TIME', 'HALSTEAD_VOLUME', 'HALSTEAD_LENGTH', 'NUM_OPERATORS', 'NUMBER_OF_LINES', 'NUM_OPERANDS', 'HALSTEAD_CONTENT', 'LOC_TOTAL', 'LOC_COMMENTS', 'LOC_EXECUTABLE', 'NUM_UNIQUE_OPERANDS', 'LOC_BLANK', 'PERCENT_COMMENTS', 'EDGE_COUNT', 'HALSTEAD_DIFFICULTY', 'NODE_COUNT', 'CONDITION_COUNT', 'NUM_UNIQUE_OPERATORS', 'LOC_CODE_AND_COMMENT', 'BRANCH_COUNT', 'MULTIPLE_CONDITION_COUNT', 'DECISION_COUNT', 'CALL_PAIRS', 'CYCLOMATIC_COMPLEXITY', 'DESIGN_COMPLE

In [38]:
total = 0
for (key,value) in w.items():
    total += value
total

195960.0